In [ ]:
### Data for this project can be found at https://drive.google.com/drive/folders/19kGE2r_m_ghKlXYqvrnSLeGS3YpQb9_i?usp=sharing

In [17]:
!pip install geopy

In [18]:
# Imports
import xml.etree.ElementTree as ET
import pandas as pd
import os
from geopy.distance import distance

## Load the timetable data and split it into meaningful dataframes


In [19]:
# Load the data

# Define namespaces
namespaces = {
    'default': 'http://www.transxchange.org.uk/'
}

# Initialize empty lists
stopPoints = []
routeLinks = []
routes = []
journeyPatternTimingLinks = []
operators = []
serviceLines = []
serviceJourneyPatterns = []
vehicleJourneys = []

# Get tag without the namespace
def tagWithoutNamespace(element):
  return element.tag.split('}')[-1] if '}' in element.tag else element.tag

# Directory containing the XML files
directory = 'G:/My Drive/MSc Project Data/timetables_data/'

# Loop through all files in the directory
for filename in os.listdir(directory):
  if filename.endswith('.xml'):
    filepath = os.path.join(directory, filename)

    # Parse the XML file
    tree = ET.parse(filepath)
    root = tree.getroot()

    # Find all StopPoints elements using XPath with namespaces and iterate through them

    for stopPoint in root.findall('.//default:StopPoints/default:AnnotatedStopPointRef', namespaces):
      record = {
          'FileName': root.get('FileName'),
          'StopPointRef': stopPoint.find('.//default:StopPointRef', namespaces).text,
          'CommonName': stopPoint.find('.//default:CommonName', namespaces).text,
          'LocalityName': stopPoint.find('.//default:LocalityName', namespaces).text
      }
      stopPoints.append(record)

    # Find RouteLinks

    for routeSection in root.findall('.//default:RouteSections/default:RouteSection', namespaces):
      routeSectionId = routeSection.get('id')
      for routeLink in routeSection.findall('.//default:RouteLink', namespaces):
        record = {
            'FileName': root.get('FileName'),
            'RouteSectionId': routeSectionId,
            'RouteLinkId': routeLink.get('id'),
            'FromStopPointRef': routeLink.find('default:From/default:StopPointRef', namespaces).text,
            'ToStopPointRef': routeLink.find('default:To/default:StopPointRef', namespaces).text,
            'StraightLineDistance': routeLink.find('default:Distance', namespaces).text,
            'Track(Longitude, Latitude)': [(float(location.find('default:Longitude', namespaces).text),
                                            float(location.find('default:Latitude', namespaces).text))
                                            for location in routeLink.findall('.default:Track/default:Mapping/default:Location', namespaces)]
        }

        routeLinks.append(record)

    # Find Routes

    for route in root.findall('.//default:Routes/default:Route', namespaces):
      record = {
          'FileName': root.get('FileName'),
          'RouteId': route.get('id'),
          'Description': route.find('.//default:Description', namespaces).text,
          'RouteSectionRef': route.find('.//default:RouteSectionRef', namespaces).text
      }
      routes.append(record)

    # Find JourneyPatternSections

    for journeyPatternSection in root.findall('.//default:JourneyPatternSections/default:JourneyPatternSection', namespaces):
      journeyPatternSectionId = journeyPatternSection.get('id')
      for journeyPatternTimingLink in journeyPatternSection.findall('.//default:JourneyPatternTimingLink', namespaces):
        fromTag = journeyPatternTimingLink.find('default:From', namespaces)
        toTag = journeyPatternTimingLink.find('default:To', namespaces)
        record = {
            'FileName': root.get('FileName'),
            'JourneyPatternSectionId': journeyPatternSectionId,
            'JourneyPatternTimingLinkId': journeyPatternTimingLink.get('id'),
            'FromStopPointRef': fromTag.find('default:StopPointRef', namespaces).text,
            'From(Id)': fromTag.get('id'),
            'From(SequenceNumber)': fromTag.get('SequenceNumber'),
            'From(TimingStatus)': fromTag.find('default:TimingStatus', namespaces).text,
            'From(FareStageNumber)': fromTag.find('default:FareStageNumber', namespaces).text,
            'From(FareStage)': fromTag.find('default:FareStage', namespaces).text,
            'ToStopPointRef': toTag.find('default:StopPointRef', namespaces).text,
            'To(Id)': fromTag.get('id'),
            'To(SequenceNumber)': toTag.get('SequenceNumber'),
            'To(TimingStatus)': toTag.find('default:TimingStatus', namespaces).text,
            'To(FareStageNumber)': toTag.find('default:FareStageNumber', namespaces).text,
            'To(FareStage)': toTag.find('default:FareStage', namespaces).text,
            'RouteLinkRef': journeyPatternTimingLink.find('default:RouteLinkRef', namespaces).text,
            'RunTime': journeyPatternTimingLink.find('default:RunTime', namespaces).text
        }

        journeyPatternTimingLinks.append(record)

    # Find Operators

    for operator in root.findall('.//default:Operators/default:Operator', namespaces):
      record = {
          'FileName': root.get('FileName'),
          'OperatorId': operator.get('id'),
          'NationalOperatorCode': operator.find('.//default:NationalOperatorCode', namespaces).text,
          'OperatorCode': operator.find('.//default:OperatorCode', namespaces).text,
          'OperatorShortName': operator.find('.//default:OperatorShortName', namespaces).text,
          'OperatorNameOnLicence': operator.find('.//default:OperatorNameOnLicence', namespaces).text,
          'LicenceNumber': operator.find('.//default:LicenceNumber', namespaces).text,
          'LicenceClassification': operator.find('.//default:LicenceClassification', namespaces).text,
          'EnquiryTelephoneNumber': operator.find('.//default:EnquiryTelephoneNumber/default:TelNationalNumber', namespaces).text,
          'ContactTelephoneNumber': operator.find('.//default:ContactTelephoneNumber/default:TelNationalNumber', namespaces).text,
          'OperatorAddresses': [[line.text for line in address] for addresses in operator.findall('.//default:OperatorAddresses', namespaces) for address in addresses],
          'Garages': [{'GarageCode': garage[0].text, 'GarageName': garage[1].text, 'Location(Longitude, Latitude)': (garage[2][0].text, garage[2][1].text)} for garage in operator.findall('.//default:Garages/default:Garage', namespaces)]
      }
      operators.append(record)

    # Find Lines

    for service in root.findall('.//default:Services/default:Service', namespaces):
      record = {
          'FileName': root.get('FileName'),
          'ServiceCode': service.find('.//default:ServiceCode', namespaces).text,
          'PrivateCode': service.find('.//default:PrivateCode', namespaces).text,
      }
      for line in service.findall('.//default:Lines/default:Line', namespaces):
        record.update({
            'LineId': line.get('id'),
            'LineName': line.find('.//default:LineName', namespaces).text,
            'OutboundDescription': {'Origin': line.find('.//default:OutboundDescription/default:Origin', namespaces).text, 'Destination': line.find('.//default:OutboundDescription/default:Destination', namespaces).text, 'Description': line.find('.//default:OutboundDescription/default:Description', namespaces).text},
            'InboundDescription': {'Origin': line.find('.//default:InboundDescription/default:Origin', namespaces).text, 'Destination': line.find('.//default:InboundDescription/default:Destination', namespaces).text, 'Description': line.find('.//default:InboundDescription/default:Description', namespaces).text},
            'OperatingPeriod': (service.find('.//default:OperatingPeriod/default:StartDate', namespaces).text, service.find('.//default:OperatingPeriod/default:EndDate', namespaces).text),
            'ServiceClassification': [tagWithoutNamespace(s) for s in service.find('.//default:ServiceClassification', namespaces)],
            'RegisteredOperatorRef': service.find('.//default:RegisteredOperatorRef', namespaces).text,
            'ServiceHasMirror': service.find('.//default:ServiceHasMirror', namespaces).text,
            'StopRequirements': [tagWithoutNamespace(s) for s in service.find('.//default:StopRequirements', namespaces)],
            'PublicUse': service.find('.//default:PublicUse', namespaces).text,
            'Express': service.find('.//default:Express', namespaces).text,
            'UseAllStopPoints': service.find('.//default:StandardService/default:UseAllStopPoints', namespaces).text,
        })
        serviceLines.append(record.copy())

    # Find Journey Patterns for each service
    for service in root.findall('.//default:Services/default:Service', namespaces):
      record = {
          'FileName': root.get('FileName'),
          'ServiceCode': service.find('.//default:ServiceCode', namespaces).text,
          'PrivateCode': service.find('.//default:PrivateCode', namespaces).text,
      }
      for journeyPattern in service.findall('.//default:StandardService/default:JourneyPattern', namespaces):
        record.update({
            'JourneyPatternId': journeyPattern.get('id'),
            'DestinationDisplay': journeyPattern.find('.//default:DestinationDisplay', namespaces).text,
            'OperatorRef': journeyPattern.find('.//default:OperatorRef', namespaces).text,
            'Direction': journeyPattern.find('.//default:Direction', namespaces).text,
            'Description': journeyPattern.find('.//default:Description', namespaces).text,
            'RouteRef': journeyPattern.find('.//default:RouteRef', namespaces).text,
            'JourneyPatternSectionRefs': journeyPattern.find('.//default:JourneyPatternSectionRefs', namespaces).text,
        })
        serviceJourneyPatterns.append(record.copy())

    # Find Vehicle Journeys

    for vehicleJourney in root.findall('.//default:VehicleJourneys/default:VehicleJourney', namespaces):
      startDeadRun = vehicleJourney.find('.//default:StartDeadRun/default:PositioningLink/default:RunTime', namespaces)
      startDeadRunTime = startDeadRun.text if startDeadRun is not None else None
      endDeadRun = vehicleJourney.find('.//default:EndDeadRun/default:PositioningLink/default:RunTime', namespaces)
      endDeadRunTime = endDeadRun.text if endDeadRun is not None else None

      record = {
          'FileName': root.get('FileName'),
          'SequenceNumber': vehicleJourney.get('SequenceNumber'),
          'Privatecode': vehicleJourney.find('.//default:PrivateCode', namespaces).text,
          'OperatorRef': vehicleJourney.find('.//default:OperatorRef', namespaces).text,
          'BlockDescription': vehicleJourney.find('.//default:Operational/default:Block/default:Description', namespaces).text,
          'BlockNumber': vehicleJourney.find('.//default:Operational/default:Block/default:BlockNumber', namespaces).text,
          'OperationalVehicleType': {'VehicleTypeCode': vehicleJourney.find('.//default:Operational/default:VehicleType/default:VehicleTypeCode', namespaces).text, 'Description': vehicleJourney.find('.//default:Operational/default:VehicleType/default:Description', namespaces).text},
          'TicketMachineServiceCode': vehicleJourney.find('.//default:Operational/default:TicketMachine/default:TicketMachineServiceCode', namespaces).text,
          'JourneyCode': vehicleJourney.find('.//default:Operational/default:TicketMachine/default:JourneyCode', namespaces).text,
          'OperatingProfile': {'RegularDayType': [tagWithoutNamespace(day) for day in vehicleJourney.find('.//default:OperatingProfile/default:RegularDayType/default:DaysOfWeek', namespaces)], 'BankHolidayOperation': [tagWithoutNamespace(day) for day in vehicleJourney.find('.//default:OperatingProfile/default:BankHolidayOperation/default:DaysOfNonOperation', namespaces)]},
          'GarageRef': vehicleJourney.find('.//default:GarageRef', namespaces).text,
          'VehicleJourneyCode': vehicleJourney.find('.//default:VehicleJourneyCode', namespaces).text,
          'ServiceRef': vehicleJourney.find('.//default:ServiceRef', namespaces).text,
          'LineRef': vehicleJourney.find('.//default:LineRef', namespaces).text,
          'JourneyPatternRef': vehicleJourney.find('.//default:JourneyPatternRef', namespaces).text,
          'StartDeadRunTime': startDeadRunTime,
          'EndDeadRunTime': endDeadRunTime,
          'DepartureTime': vehicleJourney.find('.//default:DepartureTime', namespaces).text
      }
      for day in [tagWithoutNamespace(day) for day in vehicleJourney.find('.//default:OperatingProfile/default:RegularDayType/default:DaysOfWeek', namespaces)]:
        record.update({
            'DepartureDay': day
            })
        vehicleJourneys.append(record.copy())



# Convert the list of dictionaries to a DataFrame
stopPoints_df = pd.DataFrame(stopPoints)
routeLinks_df = pd.DataFrame(routeLinks)
routes_df = pd.DataFrame(routes)
journeyPatternTimingLinks_df = pd.DataFrame(journeyPatternTimingLinks)
operators_df = pd.DataFrame(operators)
serviceLines_df = pd.DataFrame(serviceLines)
serviceJourneyPatterns_df = pd.DataFrame(serviceJourneyPatterns)
vehicleJourneys_df = pd.DataFrame(vehicleJourneys)

# List of dataframes and their names
dataframes = {
    'stopPoints_df': stopPoints_df,
    'routeLinks_df': routeLinks_df,
    'routes_df': routes_df,
    'journeyPatternTimingLinks_df': journeyPatternTimingLinks_df,
    'operators_df': operators_df,
    'serviceLines_df': serviceLines_df,
    'serviceJourneyPatterns_df': serviceJourneyPatterns_df,
    'vehicleJourneys_df': vehicleJourneys_df
}

In [20]:
# Save the dataframes as hdf files

# Define the directory path
directory_path = 'G:/My Drive/MSc Project Data/timetable_dataframes'

# Ensure the directory exists
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

# Save the DataFrames as hdf files
for name, df in dataframes.items():
    file_path = os.path.join(directory_path, name + '.h5')
    df.to_hdf(file_path, key='df', mode='w')
    print(f"Saved {name}.h5 to {file_path}")


Saved stopPoints_df.h5 to G:/My Drive/MSc Project Data/timetable_dataframes\stopPoints_df.h5
Saved routeLinks_df.h5 to G:/My Drive/MSc Project Data/timetable_dataframes\routeLinks_df.h5


C:\Users\Pc\AppData\Local\Temp\ipykernel_39248\1077184722.py:13: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['FileName', 'RouteSectionId', 'RouteLinkId', 'FromStopPointRef',
       'ToStopPointRef', 'StraightLineDistance', 'Track(Longitude, Latitude)'],
      dtype='object')]

  df.to_hdf(file_path, key='df', mode='w')


Saved routes_df.h5 to G:/My Drive/MSc Project Data/timetable_dataframes\routes_df.h5
Saved journeyPatternTimingLinks_df.h5 to G:/My Drive/MSc Project Data/timetable_dataframes\journeyPatternTimingLinks_df.h5
Saved operators_df.h5 to G:/My Drive/MSc Project Data/timetable_dataframes\operators_df.h5
Saved serviceLines_df.h5 to G:/My Drive/MSc Project Data/timetable_dataframes\serviceLines_df.h5


C:\Users\Pc\AppData\Local\Temp\ipykernel_39248\1077184722.py:13: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['FileName', 'OperatorId', 'NationalOperatorCode', 'OperatorCode',
       'OperatorShortName', 'OperatorNameOnLicence', 'LicenceNumber',
       'LicenceClassification', 'EnquiryTelephoneNumber',
       'ContactTelephoneNumber', 'OperatorAddresses', 'Garages'],
      dtype='object')]

  df.to_hdf(file_path, key='df', mode='w')
C:\Users\Pc\AppData\Local\Temp\ipykernel_39248\1077184722.py:13: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['FileName', 'ServiceCode', 'PrivateCode', 'LineId', 'LineName',
       'OutboundDescription', 'InboundDescription', 'OperatingPeriod',
       'ServiceClassification', 'RegisteredO

Saved serviceJourneyPatterns_df.h5 to G:/My Drive/MSc Project Data/timetable_dataframes\serviceJourneyPatterns_df.h5


C:\Users\Pc\AppData\Local\Temp\ipykernel_39248\1077184722.py:13: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['FileName', 'SequenceNumber', 'Privatecode', 'OperatorRef',
       'BlockDescription', 'BlockNumber', 'OperationalVehicleType',
       'TicketMachineServiceCode', 'JourneyCode', 'OperatingProfile',
       'GarageRef', 'VehicleJourneyCode', 'ServiceRef', 'LineRef',
       'JourneyPatternRef', 'StartDeadRunTime', 'EndDeadRunTime',
       'DepartureTime', 'DepartureDay'],
      dtype='object')]

  df.to_hdf(file_path, key='df', mode='w')


Saved vehicleJourneys_df.h5 to G:/My Drive/MSc Project Data/timetable_dataframes\vehicleJourneys_df.h5


In [11]:
# Display columns of each dataframe
for name, df in dataframes.items():
    print(f"Columns in {name}:")
    print(df.columns.tolist())
    print()

Columns in stopPoints_df:
['FileName', 'StopPointRef', 'CommonName', 'LocalityName']

Columns in routeLinks_df:
['FileName', 'RouteSectionId', 'RouteLinkId', 'FromStopPointRef', 'ToStopPointRef', 'StraightLineDistance', 'Track(Longitude, Latitude)']

Columns in routes_df:
['FileName', 'RouteId', 'Description', 'RouteSectionRef']

Columns in journeyPatternTimingLinks_df:
['FileName', 'JourneyPatternSectionId', 'JourneyPatternTimingLinkId', 'FromStopPointRef', 'From(Id)', 'From(SequenceNumber)', 'From(TimingStatus)', 'From(FareStageNumber)', 'From(FareStage)', 'ToStopPointRef', 'To(Id)', 'To(SequenceNumber)', 'To(TimingStatus)', 'To(FareStageNumber)', 'To(FareStage)', 'RouteLinkRef', 'RunTime']

Columns in operators_df:
['FileName', 'OperatorId', 'NationalOperatorCode', 'OperatorCode', 'OperatorShortName', 'OperatorNameOnLicence', 'LicenceNumber', 'LicenceClassification', 'EnquiryTelephoneNumber', 'ContactTelephoneNumber', 'OperatorAddresses', 'Garages']

Columns in serviceLines_df:
['Fi

## Preprocess the data

In [6]:
# Load the dataframes

stopPoints_df = pd.read_hdf('G:/My Drive/MSc Project Data/timetable_dataframes/stopPoints_df.h5', key = 'df')
routeLinks_df = pd.read_hdf('G:/My Drive/MSc Project Data/timetable_dataframes/routeLinks_df.h5', key = 'df')
routes_df = pd.read_hdf('G:/My Drive/MSc Project Data/timetable_dataframes/routes_df.h5', key = 'df')
journeyPatternTimingLinks_df = pd.read_hdf('G:/My Drive/MSc Project Data/timetable_dataframes/journeyPatternTimingLinks_df.h5', key = 'df')
operators_df = pd.read_hdf('G:/My Drive/MSc Project Data/timetable_dataframes/operators_df.h5', key = 'df')
serviceLines_df = pd.read_hdf('G:/My Drive/MSc Project Data/timetable_dataframes/serviceLines_df.h5', key = 'df')
serviceJourneyPatterns_df = pd.read_hdf('G:/My Drive/MSc Project Data/timetable_dataframes/serviceJourneyPatterns_df.h5', key = 'df')
vehicleJourneys_df = pd.read_hdf('G:/My Drive/MSc Project Data/timetable_dataframes/vehicleJourneys_df.h5', key = 'df')

In [13]:
# Calculate the track distance for each routelink

# Function to calculate the track distance
def calculate_total_distance(track):
  total_distance = 0.0
  for i in range(len(track) - 1):
    point1 = (track[i][1], track[i][0])
    point2 = (track[i+1][1], track[i+1][0])
    total_distance += distance(point1, point2).meters
  return total_distance

# Calculate the distance
routeLinks_df['TrackDistance'] = routeLinks_df['Track(Longitude, Latitude)'].apply(calculate_total_distance)

In [14]:
# Display the DataFrame with the new TrackDistance column
display(routeLinks_df[['Track(Longitude, Latitude)', 'StraightLineDistance', 'TrackDistance']])

,"Track(Longitude, Latitude)",StraightLineDistance,TrackDistance
0,"[(-1.1734874, 53.0100468), (-1.1733, 53.01004)...",515,516.579807
1,"[(-1.165984, 53.0096175), (-1.165626, 53.00961...",366,367.043676
2,"[(-1.1606512, 53.0089837), (-1.160563, 53.0089...",210,210.783498
3,"[(-1.1577676, 53.0085737), (-1.156905, 53.0084...",453,453.753422
4,"[(-1.1557027, 53.0052216), (-1.155658, 53.0051...",340,340.873306
...,...,...,...
11043,"[(-1.1677234, 52.9561233), (-1.167577, 52.9561...",301,301.998085
11044,"[(-1.16325, 52.955936), (-1.163214, 52.955936)...",374,375.038426
11045,"[(-1.1581463, 52.9561337), (-1.158125, 52.9561...",168,168.421582
11046,"[(-1.155661, 52.9559523), (-1.155115, 52.95591...",386,386.920469


In [15]:
routeLinks_df.describe(include = 'all')

,FileName,RouteSectionId,RouteLinkId,FromStopPointRef,ToStopPointRef,StraightLineDistance,"Track(Longitude, Latitude)",TrackDistance
count,11048,11048,11048,11048,11048,11048,11048,11048.000000
unique,41,342,10836,1964,1964,571,2223,NaN
top,PB0002362-26_NCT_2024-6-2.xml,79A:1,34:5_12_2147482101,3390H3,3390ME06,301,"[(-1.1457437, 52.9525577), (-1.145722, 52.9525...",NaN
freq,1537,72,2,47,47,96,42,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,341.512024
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,405.806724
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.834294
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,214.485177
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,278.198616
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,380.202683


In [16]:
# Filter the rows with the most common root link id
filtered_df = routeLinks_df[routeLinks_df['RouteLinkId'] == '34:3_1_20986'].copy()
filtered_df

,FileName,RouteSectionId,RouteLinkId,FromStopPointRef,ToStopPointRef,StraightLineDistance,"Track(Longitude, Latitude)",TrackDistance
980,PB0002362-132_NCT_2024-6-23.xml,34:3,34:3_1_20986,3390C4,3390N6,304,"[(-1.1486644, 52.9486867), (-1.148558, 52.9486...",305.514818
1192,PB0002362-132_NCT_2024-6-2.xml,34:3,34:3_1_20986,3390C4,3390N6,304,"[(-1.1486644, 52.9486867), (-1.148558, 52.9486...",305.514818


In [17]:
# Convert lists into tuples
routeLinks_df['Track(Longitude, Latitude)'] = routeLinks_df['Track(Longitude, Latitude)'].apply(tuple)
# Drop the FileName column since routelinks dont depend on filenames
routeLinks_df = routeLinks_df.drop('FileName', axis = 1)
# Drop duplicate rows based on all columns (i.e., keep only unique rows)
routeLinks_df = routeLinks_df.drop_duplicates()

routeLinks_df.describe(include = 'all')


,RouteSectionId,RouteLinkId,FromStopPointRef,ToStopPointRef,StraightLineDistance,"Track(Longitude, Latitude)",TrackDistance
count,10836,10836,10836,10836,10836,10836,10836.000000
unique,342,10836,1964,1964,571,2223,NaN
top,79A:1,89:1_1_2147483013,3390ME06,3390ME06,233,"((-1.1457437, 52.9525577), (-1.145722, 52.9525...",NaN
freq,72,1,44,47,90,42,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,341.706151
std,NaN,NaN,NaN,NaN,NaN,NaN,409.077881
min,NaN,NaN,NaN,NaN,NaN,NaN,44.834294
25%,NaN,NaN,NaN,NaN,NaN,NaN,213.974803
50%,NaN,NaN,NaN,NaN,NaN,NaN,277.735295
75%,NaN,NaN,NaN,NaN,NaN,NaN,379.547601


In [18]:
# Save the processed routeLinks dataframe
routeLinks_df.to_hdf('G:/My Drive/MSc Project Data/timetable_dataframes/routeLinks_df_processed.h5', key = 'df', mode = 'w')

C:\Users\Pc\AppData\Local\Temp\ipykernel_37832\369863355.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['RouteSectionId', 'RouteLinkId', 'FromStopPointRef', 'ToStopPointRef',
       'StraightLineDistance', 'Track(Longitude, Latitude)'],
      dtype='object')]

  routeLinks_df.to_hdf('G:/My Drive/MSc Project Data/timetable_dataframes/routeLinks_df_processed.h5', key = 'df', mode = 'w')


In [19]:
# Stop points
stopPoints_df.describe()

,FileName,StopPointRef,CommonName,LocalityName
count,3229,3229,3229,3229
unique,41,1967,1166,113
top,PB0002362-173_NCT_2024-6-2.xml,3390H3,Huntingdon Street,Nottingham
freq,150,12,20,304


In [20]:
# filter the rows with the most common stop point reference
filtered_df = stopPoints_df[stopPoints_df['StopPointRef'] == '3390H3'].copy()
filtered_df

,FileName,StopPointRef,CommonName,LocalityName
107,PB0002362-162_NCT_2024-6-2.xml,3390H3,Pryzm,Nottingham
218,PB0002362-158_NCT_2024-6-2.xml,3390H3,Pryzm,Nottingham
261,PB0002362-157_NCT_2024-6-2.xml,3390H3,Pryzm,Nottingham
391,PB0002362-132_NCT_2024-6-23.xml,3390H3,Pryzm,Nottingham
432,PB0002362-132_NCT_2024-6-2.xml,3390H3,Pryzm,Nottingham
877,PB0002362-201_NCT_2024-6-2.xml,3390H3,Pryzm,Nottingham
911,PB0002362-175_NCT_2024-6-2.xml,3390H3,Pryzm,Nottingham
945,PB0002362-189_NCT_2024-6-2.xml,3390H3,Pryzm,Nottingham
1271,PB0002362-41_NCT_2024-6-2.xml,3390H3,Pryzm,Nottingham
1739,PB0002362-200_NCT_2024-6-2.xml,3390H3,Pryzm,Nottingham


In [21]:
# Drop the FileName column since stop points dont depend on filenames
stopPoints_df = stopPoints_df.drop('FileName', axis = 1)
# Drop duplicate rows based on all columns (i.e., keep only unique rows)
stopPoints_df = stopPoints_df.drop_duplicates()

stopPoints_df.describe(include = 'all')

,StopPointRef,CommonName,LocalityName
count,1967,1967,1967
unique,1967,1166,113
top,3300GE0313,Valley Road,Clifton (Nottingham)
freq,1,6,116


In [22]:
# Routes
routes_df.describe()

,FileName,RouteId,Description,RouteSectionRef
count,353,353,353,353
unique,41,342,104,342
top,PB0002362-26_NCT_2024-6-2.xml,34C:2,34,34C:2
freq,43,2,14,2


In [23]:
# Filter the rows with the most common route id
filtered_df = routes_df[routes_df['RouteId'] == '34:3'].copy()
filtered_df

,FileName,RouteId,Description,RouteSectionRef
36,PB0002362-132_NCT_2024-6-23.xml,34:3,34,34:3
47,PB0002362-132_NCT_2024-6-2.xml,34:3,34,34:3


In [24]:
# Drop the FileName column since routes dont depend on filenames
routes_df = routes_df.drop('FileName', axis = 1)
# Drop duplicate rows based on all columns (i.e., keep only unique rows)
routes_df = routes_df.drop_duplicates()

routes_df.describe(include = 'all')

,RouteId,Description,RouteSectionRef
count,342,342,342
unique,342,104,342
top,89:1,35,89:1
freq,1,12,1


In [25]:
# Journey Pattern Timing Links
journeyPatternTimingLinks_df.describe()

,FileName,JourneyPatternSectionId,JourneyPatternTimingLinkId,FromStopPointRef,From(Id),From(SequenceNumber),From(TimingStatus),From(FareStageNumber),From(FareStage),ToStopPointRef,To(Id),To(SequenceNumber),To(TimingStatus),To(FareStageNumber),To(FareStage),RouteLinkRef,RunTime
count,48218,48218,48218,48218,48218,48218,48218,48218,48218,48218,48218,48218,48218,48218,48218,48218,48218
unique,41,1359,5044,1957,5044,72,2,48,1,1957,5044,72,2,48,1,8076,12
top,PB0002362-173_NCT_2024-6-2.xml,79:1-TDT41,JPTL1,3390J1,STPU1,10,otherPoint,18,false,3390J1,STPU1,11,otherPoint,18,false,43:1_13_2147480737,PT1M
freq,5044,72,41,233,41,1312,39369,2855,48218,233,41,1312,39363,2855,48218,33,30749


In [26]:
# Filter the rows with the most common route link references
filtered_df = journeyPatternTimingLinks_df[journeyPatternTimingLinks_df['RouteLinkRef'] == '43:1_21_2147483134'].copy()
filtered_df.head()

,FileName,JourneyPatternSectionId,JourneyPatternTimingLinkId,FromStopPointRef,From(Id),From(SequenceNumber),From(TimingStatus),From(FareStageNumber),From(FareStage),ToStopPointRef,To(Id),To(SequenceNumber),To(TimingStatus),To(FareStageNumber),To(FareStage),RouteLinkRef,RunTime
2602,PB0002362-157_NCT_2024-6-2.xml,43:1-TDT12,JPTL21,3390H3,STPU41,21,otherPoint,21,false,3390V5,STPU41,22,otherPoint,21,false,43:1_21_2147483134,PT1M
2624,PB0002362-157_NCT_2024-6-2.xml,43:1-TDT31,JPTL43,3390H3,STPU85,21,otherPoint,21,false,3390V5,STPU85,22,otherPoint,21,false,43:1_21_2147483134,PT2M
2646,PB0002362-157_NCT_2024-6-2.xml,43:1-TDT32,JPTL65,3390H3,STPU129,21,otherPoint,21,false,3390V5,STPU129,22,otherPoint,21,false,43:1_21_2147483134,PT2M
2668,PB0002362-157_NCT_2024-6-2.xml,43:1-TDT22,JPTL87,3390H3,STPU173,21,otherPoint,21,false,3390V5,STPU173,22,otherPoint,21,false,43:1_21_2147483134,PT2M
2690,PB0002362-157_NCT_2024-6-2.xml,43:1-TDT13,JPTL109,3390H3,STPU217,21,otherPoint,21,false,3390V5,STPU217,22,otherPoint,21,false,43:1_21_2147483134,PT1M


In [27]:
# Operators
operators_df.describe()

,FileName,OperatorId,NationalOperatorCode,OperatorCode,OperatorShortName,OperatorNameOnLicence,LicenceNumber,LicenceClassification,EnquiryTelephoneNumber,ContactTelephoneNumber,OperatorAddresses,Garages
count,41,41,41,41,41,41,41,41,41,41,41,41
unique,41,1,1,1,1,1,1,1,1,1,1,1
top,PB0002362-163_NCT_2024-6-2.xml,NCT,NCTR,NCT,Nottingham City Transport,Nottingham City Transport LTD,PB0002362,standardInternational,01159505745,01159505745,"[[Lower Parliament Street, Nottingham, NG1 1GG]]","[{'GarageCode': 'GOT', 'GarageName': 'Gotham G..."
freq,1,41,41,41,41,41,41,41,41,41,41,41


In [28]:
# Filter the rows with the most common operator id
filtered_df = operators_df[operators_df['OperatorId'] == 'NCT'].copy()
filtered_df.head()

,FileName,OperatorId,NationalOperatorCode,OperatorCode,OperatorShortName,OperatorNameOnLicence,LicenceNumber,LicenceClassification,EnquiryTelephoneNumber,ContactTelephoneNumber,OperatorAddresses,Garages
0,PB0002362-163_NCT_2024-6-2.xml,NCT,NCTR,NCT,Nottingham City Transport,Nottingham City Transport LTD,PB0002362,standardInternational,01159505745,01159505745,"[[Lower Parliament Street, Nottingham, NG1 1GG]]","[{'GarageCode': 'GOT', 'GarageName': 'Gotham G..."
1,PB0002362-162_NCT_2024-6-2.xml,NCT,NCTR,NCT,Nottingham City Transport,Nottingham City Transport LTD,PB0002362,standardInternational,01159505745,01159505745,"[[Lower Parliament Street, Nottingham, NG1 1GG]]","[{'GarageCode': 'GOT', 'GarageName': 'Gotham G..."
2,PB0002362-159_NCT_2024-7-4.xml,NCT,NCTR,NCT,Nottingham City Transport,Nottingham City Transport LTD,PB0002362,standardInternational,01159505745,01159505745,"[[Lower Parliament Street, Nottingham, NG1 1GG]]","[{'GarageCode': 'GOT', 'GarageName': 'Gotham G..."
3,PB0002362-158_NCT_2024-6-2.xml,NCT,NCTR,NCT,Nottingham City Transport,Nottingham City Transport LTD,PB0002362,standardInternational,01159505745,01159505745,"[[Lower Parliament Street, Nottingham, NG1 1GG]]","[{'GarageCode': 'GOT', 'GarageName': 'Gotham G..."
4,PB0002362-157_NCT_2024-6-2.xml,NCT,NCTR,NCT,Nottingham City Transport,Nottingham City Transport LTD,PB0002362,standardInternational,01159505745,01159505745,"[[Lower Parliament Street, Nottingham, NG1 1GG]]","[{'GarageCode': 'GOT', 'GarageName': 'Gotham G..."


In [29]:
# Drop the FileName column since operators dont depend on filenames
operators_df = operators_df.drop('FileName', axis = 1)

# Function to convert list of lists to tuple of tuples
def list_of_lists_to_tuple_of_tuples(lst):
    return tuple(tuple(inner) for inner in lst)

# Function to convert list of dictionaries to tuple of tuples
def list_of_dicts_to_tuple_of_tuples(lst):
    return tuple((tuple(d.items()) for d in lst))

# Convert lists into tuples to make them hashable
operators_df['OperatorAddresses'] = operators_df['OperatorAddresses'].apply(list_of_lists_to_tuple_of_tuples)
operators_df['Garages'] = operators_df['Garages'].apply(list_of_dicts_to_tuple_of_tuples)

# Drop duplicate rows based on all columns (i.e., keep only unique rows)
operators_df = operators_df.drop_duplicates()

operators_df.describe(include = 'all')

,OperatorId,NationalOperatorCode,OperatorCode,OperatorShortName,OperatorNameOnLicence,LicenceNumber,LicenceClassification,EnquiryTelephoneNumber,ContactTelephoneNumber,OperatorAddresses,Garages
count,1,1,1,1,1,1,1,1,1,1,1
unique,1,1,1,1,1,1,1,1,1,1,1
top,NCT,NCTR,NCT,Nottingham City Transport,Nottingham City Transport LTD,PB0002362,standardInternational,01159505745,01159505745,"((Lower Parliament Street, Nottingham, NG1 1GG),)","(((GarageCode, GOT), (GarageName, Gotham Garag..."
freq,1,1,1,1,1,1,1,1,1,1,1


In [30]:
# Service Lines
serviceLines_df.describe()

,FileName,ServiceCode,PrivateCode,LineId,LineName,OutboundDescription,InboundDescription,OperatingPeriod,ServiceClassification,RegisteredOperatorRef,ServiceHasMirror,StopRequirements,PublicUse,Express,UseAllStopPoints
count,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98
unique,41,40,40,96,96,56,56,5,1,1,1,1,1,1,1
top,PB0002362-135_NCT_2024-6-16.xml,PB0002362:135,PB0002362:135,NCTR:PB0002362:132:34,34,"{'Origin': 'Nottingham', 'Destination': 'Arnol...","{'Origin': 'Arnold', 'Destination': 'Nottingha...","(2024-06-02, 2024-08-31)",[NormalStopping],NCT,false,[NoNewStopsRequired],true,false,false
freq,6,6,6,2,2,7,7,79,98,98,98,98,98,98,98


In [31]:
# Filter the rows with the most common line id
filtered_df = serviceLines_df[serviceLines_df['LineId'] == 'NCTR:PB0002362:132:34'].copy()
filtered_df.head()

,FileName,ServiceCode,PrivateCode,LineId,LineName,OutboundDescription,InboundDescription,OperatingPeriod,ServiceClassification,RegisteredOperatorRef,ServiceHasMirror,StopRequirements,PublicUse,Express,UseAllStopPoints
13,PB0002362-132_NCT_2024-6-23.xml,PB0002362:132,PB0002362:132,NCTR:PB0002362:132:34,34,"{'Origin': 'Nottingham', 'Destination': 'Unive...","{'Origin': 'University Park Campus', 'Destinat...","(2024-06-23, 2024-08-31)",[NormalStopping],NCT,false,[NoNewStopsRequired],true,false,false
15,PB0002362-132_NCT_2024-6-2.xml,PB0002362:132,PB0002362:132,NCTR:PB0002362:132:34,34,"{'Origin': 'Nottingham', 'Destination': 'Unive...","{'Origin': 'University Park Campus', 'Destinat...","(2024-06-02, 2024-06-22)",[NormalStopping],NCT,false,[NoNewStopsRequired],true,false,false


In [32]:
# Drop the FileName column since service lines dont depend on filenames
serviceLines_df = serviceLines_df.drop('FileName', axis = 1)

# Function to convert dictionary to tuple
def dictToTuple(d):
  return tuple(d.items())

# Convert lists into tuples to make them hashable
serviceLines_df['InboundDescription'] = serviceLines_df['InboundDescription'].apply(dictToTuple)
serviceLines_df['OutboundDescription'] = serviceLines_df['OutboundDescription'].apply(dictToTuple)
serviceLines_df['ServiceClassification'] = serviceLines_df['ServiceClassification'].apply(tuple)
serviceLines_df['StopRequirements'] = serviceLines_df['StopRequirements'].apply(tuple)

# Drop duplicate rows based on all columns (i.e., keep only unique rows)
serviceLines_df = serviceLines_df.drop_duplicates()

serviceLines_df.describe(include = 'all') # Two rows having the same line id have different operating periods

,ServiceCode,PrivateCode,LineId,LineName,OutboundDescription,InboundDescription,OperatingPeriod,ServiceClassification,RegisteredOperatorRef,ServiceHasMirror,StopRequirements,PublicUse,Express,UseAllStopPoints
count,98,98,98,98,98,98,98,98,98,98,98,98,98,98
unique,40,40,96,96,56,56,5,1,1,1,1,1,1,1
top,PB0002362:135,PB0002362:135,NCTR:PB0002362:132:34,34,"((Origin, Nottingham), (Destination, Arnold), ...","((Origin, Arnold), (Destination, Nottingham), ...","(2024-06-02, 2024-08-31)","(NormalStopping,)",NCT,false,"(NoNewStopsRequired,)",true,false,false
freq,6,6,2,2,7,7,79,98,98,98,98,98,98,98


In [33]:
# Service Journey Patterns
serviceJourneyPatterns_df.describe() # Description means line name

,FileName,ServiceCode,PrivateCode,JourneyPatternId,DestinationDisplay,OperatorRef,Direction,Description,RouteRef,JourneyPatternSectionRefs
count,1379,1379,1379,1379,1379,1379,1379,1379,1379,1379
unique,41,40,40,1359,60,1,2,98,254,1359
top,PB0002362-135_NCT_2024-6-16.xml,PB0002362:135,PB0002362:135,34C:6-TDT1,City,NCT,inbound,79,43:1,34C:6-TDT1
freq,113,113,113,2,255,1379,757,51,33,2


In [34]:
# Filter the rows with the most common journey pattern id
filtered_df = serviceJourneyPatterns_df[serviceJourneyPatterns_df['JourneyPatternId'] == '34C:6-TDT41'].copy()
filtered_df.head()

,FileName,ServiceCode,PrivateCode,JourneyPatternId,DestinationDisplay,OperatorRef,Direction,Description,RouteRef,JourneyPatternSectionRefs
267,PB0002362-132_NCT_2024-6-23.xml,PB0002362:132,PB0002362:132,34C:6-TDT41,City Loop,NCT,inbound,34C,34C:6,34C:6-TDT41
288,PB0002362-132_NCT_2024-6-2.xml,PB0002362:132,PB0002362:132,34C:6-TDT41,City Loop,NCT,inbound,34C,34C:6,34C:6-TDT41


In [35]:
# Drop the filename because journey pattern does not depend on filename
serviceJourneyPatterns_df = serviceJourneyPatterns_df.drop('FileName', axis = 1)

# Drop duplicate rows based on all columns (i.e., keep only unique rows)
serviceJourneyPatterns_df = serviceJourneyPatterns_df.drop_duplicates()

serviceJourneyPatterns_df.describe(include = 'all') # Two rows having the same line id have different operating periods

,ServiceCode,PrivateCode,JourneyPatternId,DestinationDisplay,OperatorRef,Direction,Description,RouteRef,JourneyPatternSectionRefs
count,1359,1359,1359,1359,1359,1359,1359,1359,1359
unique,40,40,1359,60,1,2,98,254,1359
top,PB0002362:135,PB0002362:135,89:1-TDT51,City,NCT,inbound,79,43:1,89:1-TDT51
freq,113,113,1,255,1359,747,51,33,1


In [36]:
# Vehicle Journeys
vehicleJourneys_df.describe()

,FileName,SequenceNumber,Privatecode,OperatorRef,BlockDescription,BlockNumber,OperationalVehicleType,TicketMachineServiceCode,JourneyCode,OperatingProfile,GarageRef,VehicleJourneyCode,ServiceRef,LineRef,JourneyPatternRef,StartDeadRunTime,EndDeadRunTime,DepartureTime,DepartureDay
count,30936,30936,30936,30936,30936,30936,30936,30936,30936,30936,30936,30936,30936,30936,30936,3491,1675,30936,30936
unique,41,642,642,1,564,263,4,93,825,5,2,642,40,93,1338,22,16,1096,7
top,PB0002362-41_NCT_2024-6-2.xml,1,1,NCT,61040M-F,61040,"{'VehicleTypeCode': 'DD', 'Description': 'DD'}",28,1009,"{'RegularDayType': ['Monday', 'Tuesday', 'Wedn...",PSG,1,PB0002362:41,NCTR:PB0002362:58:28,41:6-TDT1,PT0S,PT20M,17:15:00,Monday
freq,1774,169,169,30936,305,354,23440,1223,281,24500,22925,169,1774,1223,436,807,575,127,4907


In [37]:
filtered_df = vehicleJourneys_df[(vehicleJourneys_df['LineRef'] == 'NCTR:PB0002362:29:6') & (vehicleJourneys_df['OperatingProfile'].apply(lambda x: 'Monday' in x['RegularDayType'])) & (vehicleJourneys_df['DepartureDay'] == 'Monday')].copy()
filtered_df

,FileName,SequenceNumber,Privatecode,OperatorRef,BlockDescription,BlockNumber,OperationalVehicleType,TicketMachineServiceCode,JourneyCode,OperatingProfile,GarageRef,VehicleJourneyCode,ServiceRef,LineRef,JourneyPatternRef,StartDeadRunTime,EndDeadRunTime,DepartureTime,DepartureDay
16878,PB0002362-29_NCT_2024-6-2.xml,1,1,NCT,1006M-F,1006,"{'VehicleTypeCode': 'DD', 'Description': 'DD'}",6,1001,"{'RegularDayType': ['Monday', 'Tuesday', 'Wedn...",PSG,1,PB0002362:29,NCTR:PB0002362:29:6,6:1-TDT11,PT18M,None,05:05:00,Monday
16883,PB0002362-29_NCT_2024-6-2.xml,2,2,NCT,3006M-F,3006,"{'VehicleTypeCode': 'DD', 'Description': 'DD'}",6,1005,"{'RegularDayType': ['Monday', 'Tuesday', 'Wedn...",PSG,2,PB0002362:29,NCTR:PB0002362:29:6,6:1-TDT11,PT18M,None,05:35:00,Monday
16888,PB0002362-29_NCT_2024-6-2.xml,3,3,NCT,1006M-F,1006,"{'VehicleTypeCode': 'DD', 'Description': 'DD'}",6,1009,"{'RegularDayType': ['Monday', 'Tuesday', 'Wedn...",PSG,3,PB0002362:29,NCTR:PB0002362:29:6,6:1-TDT1,PT5M,None,06:03:00,Monday
16893,PB0002362-29_NCT_2024-6-2.xml,4,4,NCT,3006M-F,3006,"{'VehicleTypeCode': 'DD', 'Description': 'DD'}",6,1013,"{'RegularDayType': ['Monday', 'Tuesday', 'Wedn...",PSG,4,PB0002362:29,NCTR:PB0002362:29:6,6:1-TDT1,None,None,06:33:00,Monday
16899,PB0002362-29_NCT_2024-6-2.xml,6,6,NCT,4006M-F,4006,"{'VehicleTypeCode': 'DD', 'Description': 'DD'}",6,1017,"{'RegularDayType': ['Monday', 'Tuesday', 'Wedn...",PSG,6,PB0002362:29,NCTR:PB0002362:29:6,6:1-TDT1,PT18M,None,06:48:00,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17608,PB0002362-29_NCT_2024-6-2.xml,271,271,NCT,3006M-F,3006,"{'VehicleTypeCode': 'DD', 'Description': 'DD'}",6,1222,"{'RegularDayType': ['Monday', 'Tuesday', 'Wedn...",PSG,271,PB0002362:29,NCTR:PB0002362:29:6,6:3-TDT41,None,None,22:00:00,Monday
17614,PB0002362-29_NCT_2024-6-2.xml,273,273,NCT,1006M-F,1006,"{'VehicleTypeCode': 'DD', 'Description': 'DD'}",6,1226,"{'RegularDayType': ['Monday', 'Tuesday', 'Wedn...",PSG,273,PB0002362:29,NCTR:PB0002362:29:6,6:3-TDT41,None,None,22:30:00,Monday
17621,PB0002362-29_NCT_2024-6-2.xml,276,276,NCT,3006M-F,3006,"{'VehicleTypeCode': 'DD', 'Description': 'DD'}",6,1230,"{'RegularDayType': ['Monday', 'Tuesday', 'Wedn...",PSG,276,PB0002362:29,NCTR:PB0002362:29:6,6:3-TDT41,None,None,23:00:00,Monday
17627,PB0002362-29_NCT_2024-6-2.xml,278,278,NCT,1006M-F,1006,"{'VehicleTypeCode': 'DD', 'Description': 'DD'}",6,1234,"{'RegularDayType': ['Monday', 'Tuesday', 'Wedn...",PSG,278,PB0002362:29,NCTR:PB0002362:29:6,6:3-TDT41,None,PT16M,23:30:00,Monday


In [38]:
# Filter out selected columns
vj_df = vehicleJourneys_df[['BlockNumber', 'JourneyCode', 'LineRef', 'JourneyPatternRef', 'StartDeadRunTime', 'DepartureDay', 'DepartureTime']].copy()

# Convert dead run time to seconds
vj_df['StartDeadRunTime'] = pd.to_timedelta(vj_df['StartDeadRunTime']).dt.total_seconds()

# Convert DepartureTime to datetime using a default date
vj_df['DepartureTime'] = pd.to_datetime(vj_df['DepartureTime'], format='%H:%M:%S')

# Add StartDeadRunTime to DepartureTime where StartDeadRunTime is not NaN
vj_df.loc[vj_df['StartDeadRunTime'].notna(), 'DepartureTime'] += pd.to_timedelta(vj_df['StartDeadRunTime'], unit='s')

# Extract only the time part from DepartureTime
vj_df['DepartureTime'] = vj_df['DepartureTime'].dt.time

# Remove the dead run time, once the departure time has been calculated
vj_df = vj_df.drop('StartDeadRunTime', axis = 1)

vj_df[(vj_df['LineRef'] == 'NCTR:PB0002362:29:6') & (vj_df['DepartureDay'] == 'Monday') & (vj_df['JourneyCode'] == '1009')].head()


,BlockNumber,JourneyCode,LineRef,JourneyPatternRef,DepartureDay,DepartureTime
16888,1006,1009,NCTR:PB0002362:29:6,6:1-TDT1,Monday,06:08:00


In [39]:
# Merge the journey patterns with the time taken for each journey pattern
timetable_df = pd.merge(vj_df, journeyPatternTimingLinks_df[['JourneyPatternSectionId', 'FromStopPointRef', 'From(SequenceNumber)', 'ToStopPointRef', 'To(SequenceNumber)', 'RouteLinkRef', 'RunTime']], left_on = 'JourneyPatternRef', right_on = 'JourneyPatternSectionId', how = 'left')

# Convert DepartureTime back to datetime to allow time addition
timetable_df['DepartureTime'] = pd.to_datetime(timetable_df['DepartureTime'].astype(str), format='%H:%M:%S')

# Convert RunTime to numeric (assuming RunTime is in a time format that needs conversion)
timetable_df['RunTime'] = pd.to_timedelta(timetable_df['RunTime']).dt.total_seconds()

# Change data type of sequence number
timetable_df['From(SequenceNumber)'] = timetable_df['From(SequenceNumber)'].astype(int)

# Function to update DepartureTime for each route link based on RunTime
def update_departure_times(df):
    df = df.sort_values(by='From(SequenceNumber)')
    initial_time = df.iloc[0]['DepartureTime']
    df['CumulativeRunTime'] = df['RunTime'].cumsum().shift(fill_value=0)
    df['UpdatedDepartureTime'] = df['CumulativeRunTime'].apply(lambda x: initial_time + pd.Timedelta(seconds=x))
    df['ArrivalTime'] = df['CumulativeRunTime'] + df['RunTime']
    df['ArrivalTime'] = df['ArrivalTime'].apply(lambda x: initial_time + pd.Timedelta(seconds=x))
    return df

# Apply the function to each group of JourneyPatternSection
timetable_df = timetable_df.groupby(['LineRef', 'DepartureDay', 'JourneyCode']).apply(update_departure_times).reset_index(drop=True)

# Extract the time part from UpdatedDepartureTime and ArrivalTime
timetable_df['DepartureTime'] = timetable_df['UpdatedDepartureTime'].dt.time
timetable_df['ArrivalTime'] = timetable_df['ArrivalTime'].dt.time

# Drop helper columns
timetable_df = timetable_df.drop(columns=['JourneyPatternRef', 'JourneyPatternSectionId', 'CumulativeRunTime', 'UpdatedDepartureTime'])

timetable_df.head()
timetable_df[(timetable_df['LineRef'] == 'NCTR:PB0002362:29:6') & (timetable_df['DepartureDay'] == 'Monday')].head()

,BlockNumber,JourneyCode,LineRef,DepartureDay,DepartureTime,FromStopPointRef,From(SequenceNumber),ToStopPointRef,To(SequenceNumber),RouteLinkRef,RunTime,ArrivalTime
585426,1006,1001,NCTR:PB0002362:29:6,Monday,05:23:00,3300RU0135,1,3300RU0186,2,6:1_1_2147481621,60.0,05:24:00
585427,1006,1001,NCTR:PB0002362:29:6,Monday,05:24:00,3300RU0186,2,3300RU0214,3,6:1_2_2147481620,0.0,05:24:00
585428,1006,1001,NCTR:PB0002362:29:6,Monday,05:24:00,3300RU0214,3,3300RU0222,4,6:1_3_2147481619,60.0,05:25:00
585429,1006,1001,NCTR:PB0002362:29:6,Monday,05:25:00,3300RU0222,4,3300RU0220,5,6:1_4_2147481618,0.0,05:25:00
585430,1006,1001,NCTR:PB0002362:29:6,Monday,05:25:00,3300RU0220,5,3300RU0286,6,6:1_5_2147481617,60.0,05:26:00


In [40]:
# Display a specific journey of a bus
timetable_df[(timetable_df['LineRef'] == 'NCTR:PB0002362:29:6') & (timetable_df['DepartureDay'] == 'Monday') & (timetable_df['JourneyCode'] == '1009')]

,BlockNumber,JourneyCode,LineRef,DepartureDay,DepartureTime,FromStopPointRef,From(SequenceNumber),ToStopPointRef,To(SequenceNumber),RouteLinkRef,RunTime,ArrivalTime
585495,1006,1009,NCTR:PB0002362:29:6,Monday,06:08:00,3300RU0135,1,3300RU0186,2,6:1_1_2147481621,60.0,06:09:00
585496,1006,1009,NCTR:PB0002362:29:6,Monday,06:09:00,3300RU0186,2,3300RU0214,3,6:1_2_2147481620,60.0,06:10:00
585497,1006,1009,NCTR:PB0002362:29:6,Monday,06:10:00,3300RU0214,3,3300RU0222,4,6:1_3_2147481619,60.0,06:11:00
585498,1006,1009,NCTR:PB0002362:29:6,Monday,06:11:00,3300RU0222,4,3300RU0220,5,6:1_4_2147481618,0.0,06:11:00
585499,1006,1009,NCTR:PB0002362:29:6,Monday,06:11:00,3300RU0220,5,3300RU0286,6,6:1_5_2147481617,60.0,06:12:00
585500,1006,1009,NCTR:PB0002362:29:6,Monday,06:12:00,3300RU0286,6,3300RU0322,7,6:1_6_2147481616,0.0,06:12:00
585501,1006,1009,NCTR:PB0002362:29:6,Monday,06:12:00,3300RU0322,7,3300RU0243,8,6:1_7_2147481615,120.0,06:14:00
585502,1006,1009,NCTR:PB0002362:29:6,Monday,06:14:00,3300RU0243,8,3300RU0245,9,6:1_8_2147481614,60.0,06:15:00
585503,1006,1009,NCTR:PB0002362:29:6,Monday,06:15:00,3300RU0245,9,3300RU0250,10,6:1_9_2147481613,60.0,06:16:00
585504,1006,1009,NCTR:PB0002362:29:6,Monday,06:16:00,3300RU0250,10,3300RU0249,11,6:1_10_2147481612,60.0,06:17:00


In [41]:
# Get the track route and distance based on the route link ref
timetable_merged_df = pd.merge(timetable_df, routeLinks_df[['RouteLinkId', 'Track(Longitude, Latitude)', 'TrackDistance']], how = 'left', left_on = 'RouteLinkRef', right_on = 'RouteLinkId')

# Calculate speed based on track distance and run time and also handle route links with 0 run time
timetable_merged_df['Speed'] = timetable_merged_df.apply(lambda row: row['TrackDistance'] / row['RunTime'] if row['RunTime'] != 0 else None, axis=1)

# Group by multiple columns and fill NaN values in 'Speed' with the mean speed of each group
timetable_merged_df['Speed'] = timetable_merged_df.groupby(['LineRef', 'DepartureDay', 'JourneyCode'])['Speed'].transform(
    lambda x: x.fillna(x.mean())
)

# Drop route link ref as it is the same as routelinkid
timetable_merged_df = timetable_merged_df.drop('RouteLinkRef', axis = 1)

# Get the line names based on line ref
timetable_merged_df = pd.merge(timetable_merged_df, serviceLines_df[['LineId', 'LineName']], how = 'left', left_on = 'LineRef', right_on = 'LineId')

# Remove lineid as it is same as line ref
timetable_merged_df = timetable_merged_df.drop('LineId', axis = 1)

# Save the processed timetable as an hdf file
timetable_merged_df.to_hdf('G:/My Drive/MSc Project Data/timetable_processed.h5', key='df', mode='w')


C:\Users\Pc\AppData\Local\Temp\ipykernel_37832\3179583587.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['BlockNumber', 'JourneyCode', 'LineRef', 'DepartureDay',
       'DepartureTime', 'FromStopPointRef', 'ToStopPointRef',
       'To(SequenceNumber)', 'ArrivalTime', 'RouteLinkId',
       'Track(Longitude, Latitude)', 'LineName'],
      dtype='object')]

  timetable_merged_df.to_hdf('G:/My Drive/MSc Project Data/timetable_processed.h5', key='df', mode='w')
